In [1]:
import tensorflow as tf

import tf_encrypted as tfe
from tf_encrypted.keras.losses import BinaryCrossentropy
from tf_encrypted.keras.optimizers import SGD
from tf_encrypted.player import DataOwner
from tf_encrypted.keras.datasets import LogisticArtificialDataset

config = tfe.get_config()


2022-11-01 10:06:07.633630: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-01 10:06:07.637436: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-01 10:06:07.637449: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-01 10:06:08.933126: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-01 10:06:08.933151: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

In [2]:
batch_size = 100
epochs = 20

In [3]:
train_dataset = LogisticArtificialDataset(batch_size=batch_size)
test_dataset = LogisticArtificialDataset(batch_size=batch_size, train=False)
train_client = DataOwner(config.get_player("train-client"), train_dataset.generator_builder())
test_client = DataOwner(config.get_player("test-client"), test_dataset.generator_builder())

train_iter = train_client.provide_data()
test_iter = test_client.provide_data()

## Define Model

In [4]:
model = tfe.keras.Sequential()
model.add(tfe.keras.layers.Dense(1, batch_input_shape=[batch_size, train_dataset.num_features]))
model.add(tfe.keras.layers.Activation('sigmoid'))

In [5]:
model.compile(optimizer=SGD(learning_rate=1), loss=BinaryCrossentropy())

# Train
model.fit(x=train_iter, epochs=epochs, steps_per_epoch=train_dataset.iterations)

# evaluate
model.evaluate(x=test_iter, metrics=["binary_accuracy"], steps=None)

Epoch 1/20
20/20 [==============================] - 11s 5ms/step - loss: 0.5789 - time: 0.5322
Epoch 2/20
20/20 [==============================] - 0s 5ms/step - loss: 0.5473 - time: 0.0035
Epoch 3/20
20/20 [==============================] - 0s 5ms/step - loss: 0.5204 - time: 0.0033
Epoch 4/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4974 - time: 0.0034
Epoch 5/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4777 - time: 0.0031
Epoch 6/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4605 - time: 0.0033
Epoch 7/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4456 - time: 0.0032
Epoch 8/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4324 - time: 0.0034
Epoch 9/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4205 - time: 0.0034
Epoch 10/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4099 - time: 0.0035
Epoch 11/20
20/20 [==============================] - 0s 5m

{'binary_accuracy': <tf.Tensor: shape=(), dtype=float32, numpy=0.98>}